# LangChain Basics for Email Agent
## Goal
Understand core langchain components needed for building an email agent 

## Topics
- LLM initialization
- Prompt templates
- Basic invocation

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
print("OPENAI_API_KEY exists:", bool(os.getenv("OPENAI_API_KEY")))

OPENAI_API_KEY exists: True


In [2]:
import sys
print(sys.version)

3.14.0 (v3.14.0:ebf955df7a8, Oct  7 2025, 08:20:14) [Clang 16.0.0 (clang-1600.0.26.6)]


In [3]:
import os

print("OPENAI_API_KEY exists:", bool(os.getenv("OPENAI_API_KEY")))


OPENAI_API_KEY exists: True


In [4]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')
result = llm.invoke("what is the capital of India")
print(result)

/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


content='The capital of India is New Delhi.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 13, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e413f45763', 'id': 'chatcmpl-ColImTzCl3evwwwQcxi169Oe7tBF5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b3a92-e114-7a71-8dae-af24e804fdad-0' usage_metadata={'input_tokens': 13, 'output_tokens': 8, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
